# Data Cleaner 
#### All functions related to Data Cleaning are researched, developed and tested here
#### finally past in data_cleaner.py file for scalability and faster performance in the backend.

In [274]:
import numpy as np
import pandas as pd
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor,KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings("ignore")
import seaborn  as sns
from sklearn.datasets import *

In [275]:
'''df = pd.DataFrame({
    'A': [1, 2, None, 4, 5],
    'B': [None, 3, 4, 5, 6],
    'C': [2, None, 4, 5, None],
    'D': ["Hi",None,"H ello","How","Are"],
    'E': ["2025-11-25","2005-10-24","2024-9-24","2005-10-24",None]
})'''
df=pd.read_csv("sample.csv")
df

,order_id,order_date,customer_age,is_premium,total_amount
0,1001,2024-01-15,25.0,Yes,1200.50
1,1002,2024/01/18,32.0,No,980.00
2,1003,15-02-2024,29.0,Yes,1430.75
3,1004,NaN,45.0,No,760.00
4,1005,2024-03-02,NaN,Yes,2150.00
5,1006,March 10 2024,38.0,No,NaN
6,1007,2024-03-15,27.0,Yes,1340.25


In [276]:
df.dtypes

order_id          int64
order_date          str
customer_age    float64
is_premium          str
total_amount    float64
dtype: object

## Ai validation for the datatypes of columns
For most of the columns it can give accurate results.
 if not then dtypes are asked in the Ui for confirmation and will make modifications according to it.

In [277]:
import requests

def call_llm(prompt):
    try:
        response = requests.post(
            "http://localhost:11434/api/generate",
            json={
                "model": "gemma3:4b",
                "prompt": prompt,
                "stream": False
            },
            timeout=300
        )
        
        # Check HTTP status
        if response.status_code != 200:
            print(f"Error: HTTP {response.status_code}")
            print(f"Response: {response.text}")
            return None
            
        response_json = response.json()
        
        # Check if response has the expected key
        if "response" in response_json:
            return response_json["response"]
        else:
            print(f"Error: 'response' key not found in API response")
            print(f"Available keys: {response_json.keys()}")
            print(f"Full response: {response_json}")
            return None
            
    except requests.exceptions.ConnectionError:
        print("Error: Cannot connect to Ollama at http://localhost:11434")
        print("Make sure Ollama is running: ollama serve")
        return None
    except Exception as e:
        print(f"Error: {type(e).__name__}: {e}")
        return None


In [278]:
def take_llm_decision(df):
    import json
    import re
    samples_list = {}
    column_name_list = []
    colum_before_dtype=[]
    for col in df.columns:
        column_name_list.append(col)
        samples = (df[col].astype(str).sample(5).tolist())
        samples_list[col] = samples
        colum_before_dtype.append(str(df[col].dtype))

    prompt_prefix = (
        f"Column names: {column_name_list}\n"
        f"Sample values from each column (JSON): {json.dumps(samples_list)}\n"
        f"Previous  dtype for the column: {json.dumps(colum_before_dtype)}\n"
    )

    prompt_body = (
        "You are a senior data analyst with 10 years of experience.\n"
        "Based on column names, sample values,previous data type for the column infer the most appropriate datatype.\n"
        "Do not assume the data is cleaned.\n"
        "Dates may appear as strings but should be classified as datetime.\n\n"
        "Return ONLY a valid JSON array.\n"
        "Each object must contain:\n"
        "- columnName (string)\n"
        "- datatype (one of: int, float, string, boolean, datetime, category)\n"
        "- confidence_level (number between 0 and 1)\n"
    )
    prompt_example = ('[{"columnName": "order_date", "datatype": "datetime", "confidence_level": 0.95}]')
    prompt_ = (prompt_prefix+ prompt_body+ "Example output format:\n"+ prompt_example)
    result = call_llm(prompt_)

    try:
        match = re.search(r"\[.*\]", result, re.S)
        if not match:
            return None

        data = json.loads(match.group())
        return data

    except Exception:
        return None


In [ ]:
# AI Generated

import json
import re

result = take_llm_decision(df)

if isinstance(result, str):

    sections = result.split('columnName:')[1:]  
    data = []
    
    for section in sections:
        obj = {"columnName": ""}
        lines = section.strip().split('\n')
        
        for line in lines:
            if line.startswith('datatype:'):
                obj['datatype'] = line.replace('datatype:', '').strip()
            elif line.startswith('confidence level:'):
                try:
                    obj['confidence_level'] = float(line.replace('confidence level:', '').strip())
                except:
                    obj['confidence_level'] = line.replace('confidence level:', '').strip()
            elif obj['columnName'] == '':
                obj['columnName'] = line.strip().split()[0] 
        data.append(obj)
else:
    data = result

#print(json.dumps(data, indent=2))

In [280]:
datatype = data[0]['datatype']
confidence = data[1]['confidence_level'] 

for col in data:
    print(f'''Column {col['columnName']}: {col['datatype']} (confidence: {col['confidence_level']})''')

Column order_id: string (confidence: 0.98)
Column order_date: datetime (confidence: 0.9)
Column customer_age: float (confidence: 0.95)
Column is_premium: string (confidence: 0.97)
Column total_amount: float (confidence: 0.95)


In [281]:
colums_dtype_list={}
for col in data:
    colums_dtype_list[col['columnName']]=col['datatype']
colums_dtype_list

{'order_id': 'string',
 'order_date': 'datetime',
 'customer_age': 'float',
 'is_premium': 'string',
 'total_amount': 'float'}

In [282]:
def type_casting(df,colums_dtype_list):
    for col in colums_dtype_list:
        if colums_dtype_list[col] in ("float", "integer", "int", "float64", "int64"):
            df[col] = pd.to_numeric(df[col], errors='coerce')
        elif colums_dtype_list[col] == "datetime":
            df[col] = pd.to_datetime(df[col], errors='coerce')
        elif colums_dtype_list[col]=='string':
            df[col]=df[col].astype('object').astype(str)
    return df

In [283]:
type_casting(df,colums_dtype_list)

,order_id,order_date,customer_age,is_premium,total_amount
0,1001,2024-01-15,25.0,Yes,1200.50
1,1002,NaT,32.0,No,980.00
2,1003,NaT,29.0,Yes,1430.75
3,1004,NaT,45.0,No,760.00
4,1005,2024-03-02,NaN,Yes,2150.00
5,1006,NaT,38.0,No,NaN
6,1007,2024-03-15,27.0,Yes,1340.25


In [284]:
df.dtypes

order_id                   str
order_date      datetime64[us]
customer_age           float64
is_premium                 str
total_amount           float64
dtype: object

## Data Cleaning 
### After getting the dataset with correct dtypes here starts the data cleaning with standard methods

In [285]:
categorical_columns=[]
numerical_columns=[]
for col in df.columns:
    if pd.api.types.is_numeric_dtype(df[col]):
        numerical_columns.append(col)
    else:
        categorical_columns.append(col)

In [286]:
def fill_null_values_KNN(df,numerical_columns,categorical_columns):
    
    imputer = IterativeImputer(estimator=RandomForestRegressor(),max_iter=10,random_state=42)
    df[numerical_columns] = imputer.fit_transform(df[numerical_columns])
    imputer = SimpleImputer(strategy='most_frequent')
    df[categorical_columns] = imputer.fit_transform(df[categorical_columns])
    for col in categorical_columns:
        if pd.api.types.is_datetime64_any_dtype(df[col]):
            s = pd.to_datetime(df[col], errors='coerce')
            ints = s.map(lambda x: x.value if pd.notnull(x) else np.nan).astype('float')
            ints = pd.Series(ints, index=df.index)
            ints_interp = ints.interpolate(method='linear', limit_direction='both')
            result = pd.Series([pd.NaT] * len(df), index=df.index, dtype='datetime64[ns]')
            mask = ints_interp.notna()
            if mask.any():
                result.loc[mask] = pd.to_datetime(ints_interp[mask].astype('int64'), unit='ns')
            df[col] = result
    return df

In [287]:
def standardizevalues(df,categorical_columns):
    for col in categorical_columns:
        if  not df[col].dtype=='datetime64[ns]':
            df[col] = df[col].str.strip()
            df[col] = df[col].str.replace(r'\s+', '', regex=True)
    return df

In [288]:
def remove_duplicates(df):
    df=df.drop_duplicates()
    return df

In [289]:
def remove_outliers(df):
    for col in numerical_columns:
        q1=df[col].quantile(0.25)
        q3=df[col].quantile(0.75)
        iqr=q3-q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        df = df[(df[col] > lower_bound) | (df[col] < upper_bound)]
    return df


In [290]:
df = fill_null_values_KNN(df,numerical_columns,categorical_columns)
df = standardizevalues(df,categorical_columns)
df=remove_duplicates(df)
df=remove_outliers(df)
df

,order_id,order_date,customer_age,is_premium,total_amount
0,1001,2024-01-15,25.00,Yes,1200.50
1,1002,2024-01-15,32.00,No,980.00
2,1003,2024-01-15,29.00,Yes,1430.75
3,1004,2024-01-15,45.00,No,760.00
4,1005,2024-03-02,28.08,Yes,2150.00
5,1006,2024-01-15,38.00,No,958.86
6,1007,2024-03-15,27.00,Yes,1340.25


In [291]:
type_casting(df,colums_dtype_list)

,order_id,order_date,customer_age,is_premium,total_amount
0,1001,2024-01-15,25.00,Yes,1200.50
1,1002,2024-01-15,32.00,No,980.00
2,1003,2024-01-15,29.00,Yes,1430.75
3,1004,2024-01-15,45.00,No,760.00
4,1005,2024-03-02,28.08,Yes,2150.00
5,1006,2024-01-15,38.00,No,958.86
6,1007,2024-03-15,27.00,Yes,1340.25


In [292]:
df.dtypes

order_id                   str
order_date      datetime64[ns]
customer_age           float64
is_premium                 str
total_amount           float64
dtype: object

## Creating a cleaned dataset file to send for EDA.

In [294]:
df.to_csv("cleaned_sample.csv")